In [1]:
%cd /scratch_net/biwidl214/ecetin_scratch/GSCodec

/scratch_net/biwidl214/ecetin_scratch/GSCodec


/scratch_net/biwidl214/ecetin/conda_envs/gscodec/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
from submodules.octree_generation.jit_setup import setup

setup()

Using /home/ecetin/.cache/torch_extensions/py310_cu118 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/ecetin/.cache/torch_extensions/py310_cu118/octree_generation/build.ninja...
Building extension module octree_generation...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


ninja: no work to do.


Loading extension module octree_generation...


In [3]:
from typing import Tuple
import octree_generation
import torch

class OctreeGenerator(torch.autograd.Function):
    def __init__(self):
        super(OctreeGenerator, self).__init__()

    @staticmethod
    def forward(ctx, points3D: torch.Tensor, max_depth: int) -> Tuple[torch.Tensor, torch.Tensor]:
        """Generate an octree from a set of 3D points. The octree is axis-aligned and
        the bounding box is computed from the points. The octree is generated with a
        maximum depth of `max_depth`.
        
        Args:
            points3D (torch.Tensor): A tensor of shape (N, 3) representing the 3D points.
            max_depth (int): The maximum depth of the octree.
            
        Returns:
            torch.Tensor: A tensor of shape (N, 10) representing the octant each Gaussian
                belongs to in every level of the octree.
            torch.Tensor: A tensor of shape (N, `max_depth`) representing the node ids 
                of the octree nodes that contain the points.
        """
        aabb_min = torch.min(points3D, dim=0)[0]
        aabb_max = torch.max(points3D, dim=0)[0]

        box_d = aabb_max - aabb_min
        box_min = aabb_min - 0.1 * box_d
        box_max = aabb_max + 0.1 * box_d

        # Returns point_level_bboxes, point_node_assignment
        return octree_generation.generate_octree(points3D, box_min, box_max, max_depth)

    @staticmethod
    def backward(ctx, grad_output):
        return None, None
    
def generate_octree(points: torch.Tensor, max_depth: int) -> Tuple[torch.Tensor, torch.Tensor]:
    return OctreeGenerator.apply(points, max_depth)

In [4]:
from config.build_config_spaces import ConfigReader

reader = ConfigReader("config/preset_configs/hierarchical_gaussian.yaml")
dataset = reader.dataset_config
pipeline = reader.pipeline_config
optimization = reader.optimization_config

dataset.model_path = "./output/hierarchical_model/train"
dataset.data_path += "/tandt/train"

In [5]:
dataset.model_path

'./output/hierarchical_model/train'

In [12]:
from models.splatting.base_gaussian_model import BaseGaussianModel
from training.base_gaussian_trainer import BaseGaussianTrainer
from utils.general_utils import build_scales_and_quaternions_from_cov
import os

gaussians = BaseGaussianModel(dataset=dataset)
trainer = BaseGaussianTrainer(
    dataset, optimization, pipeline, gaussians, logger=None, checkpoint=None
)
os.makedirs(f"{dataset.model_path}", exist_ok=True)
trainer.restore("output/base/truck/checkpoints/ckpt_30000.pth")

Reading camera 301/301


In [13]:
def assign_unique_values(data):
    # data should be a CUDA tensor of shape (N, 10)
    new_data = data.clone()
    N, C = data.shape
    for i in range(C):
        unique_values, new_indices = torch.unique(data[:, i], sorted=True, return_inverse=True)
        new_data[:, i] = new_indices  # Replace column with new indices

    return new_data


octree_max_depth = 10

# 3-sigma Gaussian ellipsoid surface approximation
_, gaussian_node_assignments = generate_octree(
    points=gaussians._xyz, max_depth=octree_max_depth
)
unique_per_col_assignments = assign_unique_values(gaussian_node_assignments)

In [17]:
unique_per_col_assignments[:, 0].min(), unique_per_col_assignments[:, 0].max()

(tensor(0, device='cuda:0'), tensor(7, device='cuda:0'))

In [32]:
D = 0
S = 1000
c1 = gaussians._xyz[::S][unique_per_col_assignments[::S][:, D] == 0].data
c2 = gaussians._xyz[::S][unique_per_col_assignments[::S][:, D] == 1].data
c3 = gaussians._xyz[::S][unique_per_col_assignments[::S][:, D] == 2].data
c4 = gaussians._xyz[::S][unique_per_col_assignments[::S][:, D] == 3].data
c5 = gaussians._xyz[::S][unique_per_col_assignments[::S][:, D] == 4].data
c6 = gaussians._xyz[::S][unique_per_col_assignments[::S][:, D] == 5].data
c7 = gaussians._xyz[::S][unique_per_col_assignments[::S][:, D] == 6].data
c8 = gaussians._xyz[::S][unique_per_col_assignments[::S][:, D] == 7].data

print(c1[:, 0].min(), c1[:, 0].max(), c1[:, 1].min(), c1[:, 1].max(), c1[:, 2].min(), c1[:, 2].max())
print(c2[:, 0].min(), c2[:, 0].max(), c2[:, 1].min(), c2[:, 1].max(), c2[:, 2].min(), c2[:, 2].max())
print(c3[:, 0].min(), c3[:, 0].max(), c3[:, 1].min(), c3[:, 1].max(), c3[:, 2].min(), c3[:, 2].max())
print(c4[:, 0].min(), c4[:, 0].max(), c4[:, 1].min(), c4[:, 1].max(), c4[:, 2].min(), c4[:, 2].max())
print(c5[:, 0].min(), c5[:, 0].max(), c5[:, 1].min(), c5[:, 1].max(), c5[:, 2].min(), c5[:, 2].max())   
print(c6[:, 0].min(), c6[:, 0].max(), c6[:, 1].min(), c6[:, 1].max(), c6[:, 2].min(), c6[:, 2].max())
print(c7[:, 0].min(), c7[:, 0].max(), c7[:, 1].min(), c7[:, 1].max(), c7[:, 2].min(), c7[:, 2].max())
print(c8[:, 0].min(), c8[:, 0].max(), c8[:, 1].min(), c8[:, 1].max(), c8[:, 2].min(), c8[:, 2].max())

tensor(-49.7706, device='cuda:0') tensor(1.9104, device='cuda:0') tensor(-14.3849, device='cuda:0') tensor(-8.2090, device='cuda:0') tensor(-99.1759, device='cuda:0') tensor(-6.5427, device='cuda:0')
tensor(5.5056, device='cuda:0') tensor(27.5991, device='cuda:0') tensor(-13.9600, device='cuda:0') tensor(-8.2517, device='cuda:0') tensor(-79.7989, device='cuda:0') tensor(-68.5895, device='cuda:0')
tensor(-52.4316, device='cuda:0') tensor(3.1993, device='cuda:0') tensor(-8.0927, device='cuda:0') tensor(4.7499, device='cuda:0') tensor(-83.2910, device='cuda:0') tensor(-6.4392, device='cuda:0')
tensor(3.2095, device='cuda:0') tensor(33.4579, device='cuda:0') tensor(-8.0950, device='cuda:0') tensor(6.6852, device='cuda:0') tensor(-76.8039, device='cuda:0') tensor(-6.5277, device='cuda:0')
tensor(-76.9442, device='cuda:0') tensor(2.3183, device='cuda:0') tensor(-17.4220, device='cuda:0') tensor(-8.1643, device='cuda:0') tensor(-6.1845, device='cuda:0') tensor(57.6186, device='cuda:0')
tensor

In [30]:
import torch
import plotly.graph_objects as go
import matplotlib.pyplot as plt

D = 0
S = 1000

# Example Data
points = gaussians._xyz[::S].detach().cpu()
clusters = unique_per_col_assignments[::S][:, D].detach().cpu()

def plot_3d_clusters_plotly(points, clusters):
    fig = go.Figure()

    x = points[:, 0]
    y = points[:, 1]
    z = points[:, 2]
    cluster_ids = clusters.squeeze()  # Flatten cluster tensor for easy indexing

    color_scale = plt.get_cmap('tab10')  # 'tab10' provides 10 distinct colors

    # Generate unique colors
    unique_clusters = cluster_ids.unique()
    # colors = [f'rgba({int(c)}, {int(255-c)}, {int((c/255)*100)}, 0.8)' for c in torch.linspace(0, 255, len(unique_clusters))]

    for i, cluster in enumerate(unique_clusters):
        color = f'rgba({color_scale(i)[0]*255}, {color_scale(i)[1]*255}, {color_scale(i)[2]*255}, 0.8)'
        idx = (cluster_ids == cluster)
        fig.add_trace(go.Scatter3d(
            x=x[idx].numpy(),
            y=y[idx].numpy(),
            z=z[idx].numpy(),
            mode='markers',
            marker=dict(size=5, color=color),
            name=f'Cluster {cluster.item()}'
        ))

    fig.update_layout(
        title="3D Clusters Visualization",
        scene=dict(
            xaxis_title='X Coordinate',
            yaxis_title='Y Coordinate',
            zaxis_title='Z Coordinate'
        ),
        legend_title="Cluster IDs",
        width=1200,
        height=800
    )
    fig.show()

plot_3d_clusters_plotly(points, clusters)
